<a href="https://colab.research.google.com/github/edy0710/Ingenieria-de-datos-masivos/blob/main/Evidencia/Evidencia_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importacion del dataset

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving order_products__prior.csv.zip to order_products__prior.csv.zip
Saving order_products__train.csv.zip to order_products__train.csv.zip
Saving orders.csv.zip to orders.csv.zip
Saving products.csv.zip to products.csv.zip
Saving departments.csv to departments (1).csv
Saving aisles.csv to aisles (2).csv


Descomprimir los zip

In [ ]:
import zipfile

# Descomprimir order_products__prior.csv.zip
with zipfile.ZipFile("order_products__prior.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Repetir esto para cada .zip que tengas:
with zipfile.ZipFile("orders.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile("order_products__train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile("products.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()


Leer en pandas

In [ ]:
import pandas as pd

aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products_prior = pd.read_csv('order_products__prior.csv', usecols=['order_id', 'product_id'], nrows=100000)
order_products_train = pd.read_csv("order_products__train.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")


union de datos

In [ ]:
# Unir order_products_prior con orders (para obtener user_id)
order_details = order_products_prior.merge(orders[['order_id', 'user_id']], on='order_id', how='left')

# Unir con products para obtener más información de los productos
order_details = order_details.merge(products[['product_id', 'product_name']], on='product_id', how='left')


caracteristicas por cliente

In [ ]:
# Total de productos por usuario
user_total_products = order_details.groupby('user_id')['product_id'].count().reset_index()
user_total_products.columns = ['user_id', 'total_products']

# Número de pedidos por usuario
user_total_orders = order_details.groupby('user_id')['order_id'].nunique().reset_index()
user_total_orders.columns = ['user_id', 'num_orders']

# Número de productos distintos por usuario
user_unique_products = order_details.groupby('user_id')['product_id'].nunique().reset_index()
user_unique_products.columns = ['user_id', 'unique_products']

# Promedio de productos por pedido
user_avg_products = user_total_products.merge(user_total_orders, on='user_id')
user_avg_products['avg_products_per_order'] = user_avg_products['total_products'] / user_avg_products['num_orders']

# Unir todo en un solo DataFrame
customer_features = user_avg_products.merge(user_unique_products, on='user_id')


Normalizar daros

In [ ]:
from sklearn.preprocessing import StandardScaler

features = customer_features[['total_products', 'num_orders', 'unique_products', 'avg_products_per_order']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


K-means con 4 segmentos

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)
customer_features['cluster'] = kmeans.fit_predict(features_scaled)


visualizacion

In [ ]:
# Ver algunos resultados
customer_features.groupby('cluster').mean()


,user_id,total_products,num_orders,avg_products_per_order,unique_products
cluster,,,,,
0,103733.783258,4.682241,1.000000,4.682241,4.682241
1,102256.123001,12.584563,1.000000,12.584563,12.584563
2,107266.441614,20.165605,2.055202,9.731776,17.694268
3,103591.598926,25.005372,1.005372,24.831692,24.968666


Filtracion de los segmentos

In [ ]:
# 1. Obtener IDs de usuario del segmento que quieras (por ejemplo, el clúster 0)
users_segment = customer_features[customer_features['cluster'] == 0]['user_id']

# 2. Filtrar los pedidos hechos por esos usuarios
segment_orders = orders[orders['user_id'].isin(users_segment)]

# 3. Filtrar los productos comprados en esos pedidos
segment_order_products = order_products_prior[order_products_prior['order_id'].isin(segment_orders['order_id'])]


Preparacion de los datos para apriori

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 1. Agregar los nombres de producto
basket = segment_order_products.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# 2. Agrupar productos por pedido para formar las transacciones
transactions = basket.groupby('order_id')['product_name'].apply(list).values.tolist()

# 3. Codificar las transacciones para Apriori
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

# 4. Ejecutar Apriori con soporte mínimo ajustado (puedes probar 0.01 o menos si hay pocos resultados)
frequent_itemsets = apriori(df_trans, min_support=0.02, use_colnames=True)

# 5. Obtener reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# 6. Mostrar las reglas ordenadas por lift
rules.sort_values('lift', ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
